# Introduction

This is a SQL demo. I will use [pandasql library](https://pypi.org/project/pandasql/) for use sql in pandas.

So, step one is import or install this library

In [ ]:
import pip
def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])       
import_or_install("pandasql")

In [1]:
from pandasql import PandaSQL
pdsql = PandaSQL()

Ok, now we need data. We will use one Seaborn dataset, from Titanic survivors.

In [2]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Ok, 'head()' function show the first 5 rows (by default), so let doing the same, but with sql query:

In [3]:
pdsql("SELECT * FROM titanic LIMIT 5;")

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,1,None,Southampton,no,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,0,C,Cherbourg,yes,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,0,None,Southampton,yes,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,0,C,Southampton,yes,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,1,None,Southampton,no,1


We can, of course, select some condition:

In [4]:
pdsql("SELECT * FROM titanic WHERE sex='female' LIMIT 5;")

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,1,1,female,38.0,1,0,71.2833,C,First,woman,0,C,Cherbourg,yes,0
1,1,3,female,26.0,0,0,7.9250,S,Third,woman,0,None,Southampton,yes,1
2,1,1,female,35.0,1,0,53.1000,S,First,woman,0,C,Southampton,yes,0
3,1,3,female,27.0,0,2,11.1333,S,Third,woman,0,None,Southampton,yes,0
4,1,2,female,14.0,1,0,30.0708,C,Second,child,0,None,Cherbourg,yes,0


Given that pdsql return a dataframe, we can use other dataframe functions over it. For simple example, call head with just 2 rows:

In [5]:
pdsql("SELECT * FROM titanic WHERE sex='female' LIMIT 5;").head(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,1,1,female,38.0,1,0,71.2833,C,First,woman,0,C,Cherbourg,yes,0
1,1,3,female,26.0,0,0,7.9250,S,Third,woman,0,None,Southampton,yes,1


... or tail from the first 5 rows:

In [6]:
pdsql("SELECT * FROM titanic WHERE sex='female' LIMIT 5;").tail(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
3,1,3,female,27.0,0,2,11.1333,S,Third,woman,0,None,Southampton,yes,0
4,1,2,female,14.0,1,0,30.0708,C,Second,child,0,None,Cherbourg,yes,0


## Agregate queries

Let explore how many people

In [7]:
query = """
SELECT count(*) AS count, class, sex, who, survived, count(survived) as count_survived
FROM titanic
GROUP BY class, sex, who, survived
ORDER BY class, sex, who DESC
"""
pdsql(query)

,count,class,sex,who,survived,count_survived
0,2,First,female,woman,0,2
1,89,First,female,woman,1,89
2,1,First,female,child,0,1
3,2,First,female,child,1,2
4,77,First,male,man,0,77
5,42,First,male,man,1,42
6,3,First,male,child,1,3
7,6,Second,female,woman,0,6
8,60,Second,female,woman,1,60
9,10,Second,female,child,1,10


In [8]:
query = """
SELECT count(*) AS count, class, sex, who, survived, count(survived) as count_survived
FROM titanic
GROUP BY class, sex, who, survived
HAVING (survived)>0
ORDER BY class, sex, who DESC
"""
survived = pdsql(query)
survived

,count,class,sex,who,survived,count_survived
0,89,First,female,woman,1,89
1,2,First,female,child,1,2
2,42,First,male,man,1,42
3,3,First,male,child,1,3
4,60,Second,female,woman,1,60
5,10,Second,female,child,1,10
6,8,Second,male,man,1,8
7,9,Second,male,child,1,9
8,56,Third,female,woman,1,56
9,16,Third,female,child,1,16


Ok, let check if that is correct: we have a dataframe, called 'survived' with agregational data. We can use '<>column.sum()' to sum all element in that column:

In [9]:
survived.count_survived.sum()

342

We can see that it the same with the simple query

In [10]:
query = """
SELECT count(*)
FROM titanic
WHERE (survived)>0
ORDER BY class, sex, who DESC
"""
pdsql(query)

,count(*)
0,342


## Separate classes

In [11]:
query = """
SELECT *
FROM titanic
WHERE class='First'
"""
c1 = pdsql(query)
c1

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,1,1,female,38.0,1,0,71.2833,C,First,woman,0,C,Cherbourg,yes,0
1,1,1,female,35.0,1,0,53.1000,S,First,woman,0,C,Southampton,yes,0
2,0,1,male,54.0,0,0,51.8625,S,First,man,1,E,Southampton,no,1
3,1,1,female,58.0,0,0,26.5500,S,First,woman,0,C,Southampton,yes,1
4,1,1,male,28.0,0,0,35.5000,S,First,man,1,A,Southampton,yes,1
5,0,1,male,19.0,3,2,263.0000,S,First,man,1,C,Southampton,no,0
6,0,1,male,40.0,0,0,27.7208,C,First,man,1,None,Cherbourg,no,1
7,1,1,female,NaN,1,0,146.5208,C,First,woman,0,B,Cherbourg,yes,0
8,0,1,male,28.0,1,0,82.1708,C,First,man,1,None,Cherbourg,no,0
9,0,1,male,42.0,1,0,52.0000,S,First,man,1,None,Southampton,no,0


In [12]:
query = """
SELECT *
FROM titanic
WHERE class='Second'
"""
c2 = pdsql(query)
c2

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,1,2,female,14.00,1,0,30.0708,C,Second,child,0,None,Cherbourg,yes,0
1,1,2,female,55.00,0,0,16.0000,S,Second,woman,0,None,Southampton,yes,1
2,1,2,male,NaN,0,0,13.0000,S,Second,man,1,None,Southampton,yes,1
3,0,2,male,35.00,0,0,26.0000,S,Second,man,1,None,Southampton,no,1
4,1,2,male,34.00,0,0,13.0000,S,Second,man,1,D,Southampton,yes,1
5,0,2,male,66.00,0,0,10.5000,S,Second,man,1,None,Southampton,no,1
6,0,2,female,27.00,1,0,21.0000,S,Second,woman,0,None,Southampton,no,0
7,1,2,female,3.00,1,2,41.5792,C,Second,child,0,None,Cherbourg,yes,0
8,1,2,female,29.00,1,0,26.0000,S,Second,woman,0,None,Southampton,yes,0
9,1,2,female,21.00,0,0,10.5000,S,Second,woman,0,None,Southampton,yes,1


In [13]:
query = """
SELECT *
FROM titanic
WHERE class='Third'
"""
c3 = pdsql(query)
c3

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,1,None,Southampton,no,0
1,1,3,female,26.0,0,0,7.9250,S,Third,woman,0,None,Southampton,yes,1
2,0,3,male,35.0,0,0,8.0500,S,Third,man,1,None,Southampton,no,1
3,0,3,male,NaN,0,0,8.4583,Q,Third,man,1,None,Queenstown,no,1
4,0,3,male,2.0,3,1,21.0750,S,Third,child,0,None,Southampton,no,0
5,1,3,female,27.0,0,2,11.1333,S,Third,woman,0,None,Southampton,yes,0
6,1,3,female,4.0,1,1,16.7000,S,Third,child,0,G,Southampton,yes,0
7,0,3,male,20.0,0,0,8.0500,S,Third,man,1,None,Southampton,no,1
8,0,3,male,39.0,1,5,31.2750,S,Third,man,1,None,Southampton,no,0
9,0,3,female,14.0,0,0,7.8542,S,Third,child,0,None,Southampton,no,1


### There is some 'embark_town' common in c1 and c2?

In [14]:
query = """
SELECT DISTINCT c1.embark_town
FROM c1, c2
WHERE c1.embark_town = c2.embark_town
--LIMIT 2
"""
pdsql(query)


,embark_town
0,Cherbourg
1,Southampton
2,Queenstown


### There is some 'embark_town' common in c1, c2 and c3?

In [15]:
query = """
SELECT DISTINCT c1.embark_town
FROM c1, c2, c3
WHERE 
c1.embark_town = c2.embark_town and
c2.embark_town = c3.embark_town 
--LIMIT 2
"""
pdsql(query)


,embark_town
0,Cherbourg
1,Southampton
2,Queenstown


Just for check, let see the shared from c1 and c3...

In [17]:
query = """
SELECT DISTINCT c1.embark_town
FROM c1, c3
WHERE c1.embark_town = c3.embark_town
"""
pdsql(query)


,embark_town
0,Cherbourg
1,Southampton
2,Queenstown


Yes, the 3 classes have 'embark_town' in Cherbourg, Southampton and Queenstown.

# [See the interactive version](https://nbviewer.jupyter.org/github/NeoRichard/DemoSQL/blob/master/DemoSQL1.ipynb)